In [31]:
from langchain_community.llms import Ollama
from langchain.vectorstores import Weaviate
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory

In [32]:
loader = PyPDFLoader('./data/Cognitive Behavioral Therapy Made Simple 10 Strategies for Managing Anxiety, Depression, Anger, Panic, and Worry (Seth J. Gillihan) (Z-Library).pdf')
data = loader.load()

In [33]:
print(f'Number of pages: {len(data)}')

Number of pages: 191


In [34]:
text_gen = ''
for page in data:
    text_gen += page.page_content
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap = 5,
    keep_separator=False,
    separators='.',
)

In [35]:
text_gen

"Copyright © 2018 by Seth J. Gillihan\nNo part of this publication may be reproduced, stored in a retrieval system, or transmitted in any\nform or by any means, electronic, mechanical, photocopying, recording, scanning, or otherwise,\nexcept as permitted under Sections 107 or 108 of the 1976 United States Copyright Act, without\nthe prior written permission of the Publisher . Requests to the Publisher for permission should be\naddressed to the Permissions Department, Althea Press, 6005 Shellmound Street, Suite 175,\nEmeryville, CA 94608.\nLimit of Liability/Disclaimer of W arranty: The Publisher and the author make no\nrepresentations or warranties with respect to the accuracy or completeness of the contents of this\nwork and specifically disclaim all warranties, including without limitation warranties of fitness\nfor a particular purpose. No warranty may be created or extended by sales or promotional\nmaterials. The advice and strategies contained herein may not be suitable for every 

In [36]:
chunks = text_splitter.split_text(text_gen)
print(f'Number of chunks: {len(chunks)}')

Number of chunks: 329


In [37]:
chunks[30:35]

['Prescribing doctors will\ntake into account the potential benefits of an SSRI or benzodiazepine\nversus the common side effects.\nThis book focuses on CBT rather than medication. Check with your\nprimary care doctor or a psychiatrist if you’re interested in a medication\nconsultation.\nHow Can You Help Yourself?\nFor CBT to work best, it’s necessary to zero in on your particular needs.\nAre you struggling with low mood, an uncontrolled temper, pervasive\nworry, or something else? Let’s consider how CBT can be used to address\ndifferent conditions and allow you to help yourself through the particular\nissues that you are facing.\nDEPRESSION\nThoughts, feelings, and behaviors work together in a downward spiral when\nwe’re depressed. Low mood and poor motivation make it hard to find\npleasure even in things we used to enjoy. We see the world and ourselves in\na negative light. As our thoughts and mood darken, we’re likely to pull back\nfrom many of our activities, further deepening our 

In [38]:
document_gen = [Document(page_content=t) for t in chunks]
document_gen[30:35]

[Document(page_content='Prescribing doctors will\ntake into account the potential benefits of an SSRI or benzodiazepine\nversus the common side effects.\nThis book focuses on CBT rather than medication. Check with your\nprimary care doctor or a psychiatrist if you’re interested in a medication\nconsultation.\nHow Can You Help Yourself?\nFor CBT to work best, it’s necessary to zero in on your particular needs.\nAre you struggling with low mood, an uncontrolled temper, pervasive\nworry, or something else? Let’s consider how CBT can be used to address\ndifferent conditions and allow you to help yourself through the particular\nissues that you are facing.\nDEPRESSION\nThoughts, feelings, and behaviors work together in a downward spiral when\nwe’re depressed. Low mood and poor motivation make it hard to find\npleasure even in things we used to enjoy. We see the world and ourselves in\na negative light. As our thoughts and mood darken, we’re likely to pull back\nfrom many of our activities, 

In [39]:
embedding_model = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-small-en-v1.5')

/home/chescore/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/chescore/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/chescore/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [41]:
vector_store = Weaviate.from_documents(document_gen, 
                                    embedding_model, 
                                    weaviate_url = 'http://localhost:8080'
)

/home/chescore/anaconda3/lib/python3.11/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.1.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [42]:
retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

In [43]:
query_example = 'How can one deal with depression?'
similarity_search_results = vector_store.similarity_search(query_example)
print(similarity_search_results[0].page_content)

Prescribing doctors will
take into account the potential benefits of an SSRI or benzodiazepine
versus the common side effects.
This book focuses on CBT rather than medication. Check with your
primary care doctor or a psychiatrist if you’re interested in a medication
consultation.
How Can You Help Yourself?
For CBT to work best, it’s necessary to zero in on your particular needs.
Are you struggling with low mood, an uncontrolled temper, pervasive
worry, or something else? Let’s consider how CBT can be used to address
different conditions and allow you to help yourself through the particular
issues that you are facing.
DEPRESSION
Thoughts, feelings, and behaviors work together in a downward spiral when
we’re depressed. Low mood and poor motivation make it hard to find
pleasure even in things we used to enjoy. We see the world and ourselves in
a negative light. As our thoughts and mood darken, we’re likely to pull back
from many of our activities, further deepening our depression


In [44]:
llm = Ollama(model='mistral:7b-instruct-q4_K_M', temperature=0.5)

In [45]:
prompt_template = """
   ### [INST]
   Instruction: You are a therapy chatbot that empathizes with its clients and tries to make them feel better about themselves.
   Use kind words like 'Sorry' or 'I apologise' to show that you sympathise with them. 
   Example: 
   Client: I don't know how to deal with depression
   You(The bot's response): I am sorry to hear that. Here are some solutions that might help: {context}
   
   If certain statements are vague, try asking questions to the clients.
   Example:
   Client: I feel sad.
   You(The bot's response): I am sorry to hear that. What might be the cause of that?
   
   Here is context to help: {context}
   Follow-up Instructions:
   {chat_history}
   ##QUESTION:
   {question}
   [/INST]
"""

prompt = PromptTemplate(
    input_variables=['context','question', 'chat_history'],
    template=prompt_template
)

In [46]:
memory = ConversationBufferMemory(llm=llm, memory_key='chat_history', output_key='answer', return_messages=True)
chain = ConversationalRetrievalChain.from_llm(llm=llm, 
                                        memory=memory,
                                        retriever=retriever,
                                        combine_docs_chain_kwargs={'prompt': prompt}
)

In [47]:
query = 'I am dealing with anxiety. How can I deal with this?' 
chain.invoke(query)

{'question': 'I am dealing with anxiety. How can I deal with this?',
 'chat_history': [HumanMessage(content='I am dealing with anxiety. How can I deal with this?'),
  AIMessage(content='\nAnxiety is a common experience that can be difficult to manage. It\'s important to remember that while it may feel overwhelming at times, there are many strategies and tools you can use to help manage your anxiety. Here are some tips for dealing with anxiety:\n\n1. Practice mindfulness: Mindfulness involves being present in the moment and focusing on your thoughts, feelings, and sensations without judgment. This can be a helpful way to manage anxiety by allowing you to observe your thoughts without getting caught up in them. There are many ways to practice mindfulness, such as meditation, deep breathing exercises, or simply taking time to focus on your senses.\n2. Challenge negative thoughts: One of the most effective ways to manage anxiety is to challenge negative thoughts that may be contributing to